In [6]:
import pandas as pd
import json
from jinja2 import Template
import pdfkit

In [7]:
with open('db_con.txt') as a:
    db_con = a.read()

In [8]:
proposals = pd.read_pickle('proposals.pkl')
recipients = pd.read_sql('select * from recipients', con=db_con, index_col='id')
themes = pd.read_pickle('themes.pkl')

In [9]:
themes

,proposal_id,parent_name
0,1,Education and training
1,1,Public and societal benefit
2,2,Health and medicine
3,2,Public and societal benefit
5,2,Social welfare
...,...,...
115521,40772,Employment
115522,40772,Education and training
115524,40773,Social welfare
115526,40773,Health and medicine


In [10]:
def output_template(data):
    with open('report_template.html.j2') as a, open('docs/assets/style.css') as stylesheet, \
        open('docs/assets/blue-logo.svg') as logo:
        template = Template(a.read())
        return template.render(**data, stylesheet=stylesheet.read(), logo=logo.read())

In [16]:
r_id = 136
r = recipients[recipients.index==r_id].iloc[0]
ps = proposals[proposals.recipient_id==r_id].reset_index().to_dict(orient='records')
for p in ps:
    p['themes'] = themes.loc[themes.proposal_id==p['id'], 'parent_name'].to_list()
pdfkit.from_string(
    output_template({
        'recipient': r.to_dict(),
        'proposals': ps,
    }),
    "sample_report.pdf"
)
ps

OSError: wkhtmltopdf reported an error:
Loading pages (1/6)
[>                                                           ] 0%[======>                                                     ] 10%Warning: Blocked access to file                                   
[=====================>                                      ] 36%Error: Failed to load about:blank, with network status code 301 and http status code 0 - Protocol "about" is unknown
[========================>                                   ] 41%[===========================>                                ] 45%[=====================================>                      ] 62%[============================================>               ] 74%[============================================================] 100%Counting pages (2/6)                                               
[============================================================] Object 1 of 1Resolving links (4/6)                                                  

In [14]:
proposals.loc[26652, :]

recipient_id                                          19881
title                                                  None
description                                            None
min_duration                                            NaN
min_amount                                              NaN
created_at                       2017-08-07 11:59:19.144606
updated_at                       2018-10-31 23:32:14.281612
state                                               invalid
prevent_funder_verification                            None
category_code                                           101
public_consent                                         None
support_details                        No description given
max_amount                                              NaN
max_duration                                            NaN
geographic_scale                                    missing
user_id                                                3169
access_token                         ANo

In [27]:
recipients proposals[['recipient_name',
       'recipient_description', 'recipient_category_code',
       'recipient_operating_for', 'recipient_income_band', 'recipient_category']].drop_duplicates()

,recipient_name,recipient_description,recipient_category_code,recipient_operating_for,recipient_income_band,recipient_category
id,,,,,,
37737,Fingringhoe Church Green Trust,Charity registered in England and Wales: Compa...,301.0,4 years or more,£10k - £99k,A charitable organisation
37755,The Well Foundation,Charity registered in Scotland,301.0,4 years or more,£10k - £99k,A charitable organisation
37776,2nd Tidworth Brownies,,201.0,Less than 3 years,Less than £10k,A community or voluntary group
37793,,,101.0,NaN,NaN,An individual
37794,Bath hospital radio,CIO,301.0,4 years or more,Less than £10k,A charitable organisation
...,...,...,...,...,...,...
37732,Whitecroft sports ground trust,,201.0,4 years or more,Less than £10k,A community or voluntary group
37733,The safetonet foundation,"Company Limited by Guarantee (CLG), registered...",301.0,Less than 12 months,Less than £10k,A charitable organisation
37734,Cohesion & integration network,charity registered in england & wales,301.0,Less than 12 months,Less than £10k,A charitable organisation


In [11]:
proposals.iloc[0].to_dict()

{'recipient_id': 39547,
 'title': 'Increasing the provision of sheltered homes for the elderly in Fingringhoe',
 'description': 'We wish to increase our provision of sheltered homes for the elderly or disabled living in Fingringhoe or the surrounding villages who wish to remain living in Fingringhoe with support. ',
 'min_duration': 6.0,
 'min_amount': 90000.0,
 'created_at': Timestamp('2018-12-03 18:12:38.038646'),
 'updated_at': Timestamp('2018-12-03 18:12:38.038646'),
 'state': 'complete',
 'prevent_funder_verification': None,
 'category_code': 201,
 'public_consent': True,
 'support_details': '',
 'max_amount': 120000.0,
 'max_duration': 9.0,
 'geographic_scale': 'local',
 'user_id': 18421.0,
 'access_token': 'R9rXvV-Ftim0pCZvxqfCGw',
 'private': NaT,
 'collection_id': 195.0,
 'collection_type': 'Funder',
 'duplicate_of': nan,
 'migrated_on': NaT,
 'assessments_count': 2,
 'recipient_name': 'Fingringhoe Church Green Trust',
 'recipient_description': 'Charity registered in England a

In [3]:
region_order = [
    'North East',
    'North West',
    'Yorkshire and The Humber',
    'East Midlands',
    'West Midlands',
    'East of England',
    'London',
    'South East',
    'South West',
    'England',
    'Scotland',
    'Wales',
    'Northern Ireland',
    'UK wide',
    'Overseas',
    'UK and overseas',
]

In [20]:
def create_profile(p, t):
    areas = p['area'].value_counts().to_dict()
    return {
        'recipient_category': p['recipient_category'].value_counts().to_dict(),
        'recipient_income_band': p['recipient_income_band'].value_counts().sort_index().to_dict(),
        'recipient_operating_for': p['recipient_operating_for'].value_counts().sort_index().to_dict(),
        'geographic_scale': p['geographic_scale'].value_counts().to_dict(),
        'category': p['category'].value_counts().to_dict(),
        'amount_stats': p['min_amount'].describe().to_dict(),
        'amount_bins': p['amount_bins'].value_counts().sort_index().to_dict(),
        'duration_bins': p['duration_bins'].value_counts().sort_index().to_dict(),
        'area': {
            r: areas.get(r, 0)
            for r in region_order
        },
        'themes': t[t['proposal_id'].isin(p.index)]['parent_name'].value_counts().to_dict(),
        'examples': p.loc[
            (p.created_at.dt.year >= 2018) & (p.recipient_category != "An individual"),
            ["title", "description", 'geographic_scale', 
            'recipient_operating_for', 'recipient_income_band', 
            'area', 'category', 'recipient_category', 
            'amount_bins', 'duration_bins']
        ].sample(15).to_dict(orient='records'),
    }

In [21]:
t = themes[themes['parent_name']!="Groups"]
results = {
    "area": {},
    "theme": {},
}

# full dataset
results['all'] = create_profile(proposals, t)

# by area
for k in proposals.area.dropna().unique():
    print(k)
    results['area'][k] = create_profile(proposals[proposals.area==k], t)

# by theme
for k in t['parent_name'].unique():
    print(k)
    p = proposals.loc[themes.loc[themes['parent_name']==k, 'proposal_id'], :]
    results['theme'][k] = create_profile(p, t)
    
with open('docs/results.json'.format(k), 'w') as a:
    json.dump(results, a, indent=4)

East of England
Scotland
South West
West Midlands
North West
Overseas
North East
England
South East
UK and overseas
East Midlands
Wales
Yorkshire and The Humber
London
UK wide
Northern Ireland
Education and training
Public and societal benefit
Health and medicine
Social welfare
Crime and justice
Employment
Climate and the environment
Community improvement and capacity building
International and foreign affairs
Arts and recreation
Science and technology


In [14]:
proposals.loc[themes.loc[themes['parent_name']=="Groups", 'proposal_id'], :]

,recipient_id,title,description,min_duration,min_amount,created_at,updated_at,state,prevent_funder_verification,category_code,...,recipient_name,recipient_description,recipient_category_code,recipient_operating_for,recipient_income_band,area,category,recipient_category,amount_bins,duration_bins
id,,,,,,,,,,,,,,,,,,,,,
5,20166,Futsal Club Enfield in the Community,Community Futsal Team who works with other in ...,36.0,20.0,2015-12-15 12:50:46.418814,2018-10-31 23:36:34.897005,complete,False,101,...,Futsal club Enfield,None,102.0,NaN,NaN,NaN,Other,Another type of organisation,Under £500,3 years
6,20172,Costs for management and administration of an ...,The programme funding is from a private donor ...,48.0,400000.0,2015-12-15 14:20:38.359748,2018-10-31 23:36:40.129029,complete,False,203,...,Oldham Enterprise Trust,None,102.0,NaN,NaN,North West,Revenue - Project,Another type of organisation,£100k - £1m,4 years +
9,20184,jul,cxvbxfb,12.0,40000.0,2015-12-15 22:28:35.279094,2018-10-31 23:36:55.494343,complete,False,101,...,Trans,None,102.0,NaN,NaN,NaN,Other,Another type of organisation,£10k - £100k,12 months
14,20204,Dr of Divinity,To work full-time on a 3 year commitment to ob...,36.0,120000.0,2015-12-16 18:27:16.072643,2018-10-31 23:37:09.020890,complete,False,101,...,Doctorate in Theology,None,102.0,NaN,NaN,Overseas,Other,Another type of organisation,£100k - £1m,3 years
20,20235,Building refurbishment and supplementary educa...,"To use funding to refurbish the building, for ...",12.0,25000.0,2015-12-18 18:33:41.278296,2018-10-31 23:37:33.093693,incomplete,True,203,...,Rock-I ministries LTD,None,202.0,Less than 3 years,£10k - £99k,England,Revenue - Project,An unincorporated association,£10k - £100k,12 months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40764,45350,Funding for Music/instrumental lessons for ch...,The Swindon music trust has now been running f...,1.0,2500.0,2020-06-16 12:18:27.443797,2020-06-16 12:18:27.443797,complete,None,202,...,Swindon Music Trust,Charity registered in England & Wale,301.0,4 years or more,Less than £10k,NaN,Revenue - Core,A charitable organisation,£2k - £5k,Under 12 months
40765,45352,"""Listening Ear"" Phone line supporting adult su...",To provide a phone line manned by trained volu...,6.0,12000.0,2020-06-16 14:52:04.976067,2020-06-16 14:52:04.976067,complete,None,203,...,The Maggie Oliver Foundation,CIO,301.0,Less than 12 months,£10k - £99k,NaN,Revenue - Project,A charitable organisation,£10k - £100k,Under 12 months
40767,45360,Engaging the public with health research,A research volunteers co-designed website to i...,12.0,10000.0,2020-06-23 08:41:00.155344,2020-06-23 08:41:00.155344,complete,None,203,...,Portsmouth Hospitals NHS Trust,NHS Trust,301.0,4 years or more,£1m - £10m,South East,Revenue - Project,A charitable organisation,£5k - £10k,12 months


In [16]:
pdfkit.from_url(
    'http://127.0.0.1:5000/report/136',
    "sample_report.pdf"
)

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


True

In [22]:
for r in proposals['recipient_id'].sample(10).unique():
    pdfkit.from_url(
        'http://127.0.0.1:5000/report/{}'.format(r),
        "reports/beehive_legacy_{}.pdf".format(r)
    )

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                         

In [28]:
recip = pd.read_sql('''
select distinct p.recipient_id,
    r.name as "recipient_name",
    u.first_name,
    u.last_name,
    u.email,
    u.last_seen,
    u.agree_to_terms 
from proposals p 
    inner join recipients r 
        on p.recipient_id = r.id 
    inner join users u
        on p.user_id = u.id
''', con=db_con, index_col='recipient_id')
recip

,recipient_name,first_name,last_name,email,last_seen,agree_to_terms
recipient_id,,,,,,
16650,Movivo,Alasdair,Chesney,a.chesney@movivo.mobi,NaT,True
43108,Democracy Volunteers,John,Ault,john.ault@democracyvolunteers.org,NaT,None
41343,Disability action in the borough of Barnet,Jeffrey,Raphael,dabbtrustees@gmail.com,2017-09-04 15:48:36.916681,True
22984,Kitoko Ltd,Benjamin,Mubiayi,benjamin@kitoko.co.uk,2016-04-14 00:20:15.031987,True
26787,Community Emergency Response Team Limited,Saj,Ghafoor,info@edenfloodvolunteers.org.uk,2016-11-14 11:34:30.762489,True
...,...,...,...,...,...,...
33118,Kag Advocacy Cio,Sue,Boakes,rights@kag.org.uk,2017-12-11 14:23:00.285931,True
40621,Cranleigh Cricket Club,Karen,Botha,karenbotha13@gmail.com,NaT,None
44731,Development Initiatives Poverty Research Limited,Jennie,Robertson,jennie.robertson@devinit.org,NaT,None


In [30]:
recip.index.unique()

Int64Index([16650, 43108, 41343, 22984, 26787, 22042, 35819, 18665, 40928,
            24384,
            ...
            33135, 38530, 23073, 33121, 32107, 33118, 40621, 44731, 43917,
            33938],
           dtype='int64', name='recipient_id', length=21477)

In [32]:
recip.loc[16650, :]

recipient_name                   Movivo
first_name                     Alasdair
last_name                       Chesney
email             a.chesney@movivo.mobi
last_seen                           NaT
agree_to_terms                     True
Name: 16650, dtype: object

In [33]:
help(pdfkit.from_url)

Help on function from_url in module pdfkit.api:

from_url(url, output_path, options=None, toc=None, cover=None, configuration=None, cover_first=False)
    Convert file of files from URLs to PDF document
    
    :param url: URL or list of URLs to be saved
    :param output_path: path to output PDF file. False means file will be returned as string.
    :param options: (optional) dict with wkhtmltopdf global and page options, with or w/o '--'
    :param toc: (optional) dict with toc-specific wkhtmltopdf options, with or w/o '--'
    :param cover: (optional) string with url/filename with a cover html page
    :param configuration: (optional) instance of pdfkit.configuration.Configuration()
    :param configuration_first: (optional) if True, cover always precedes TOC
    
    Returns: True on success

